# Rules to modules

*WIP*

In [2]:
from neuralogic.core import Atom, Template, Var, Term, Backend
from neuralogic.core.settings import Settings, Optimizer
from neuralogic.utils.data import Dataset


from neuralogic.utils.templates.translator import translate_rules
from neuralogic.utils.templates import TemplateList

## Simple GCN (one layer)

In [3]:
with Template().context() as template:
    template.add_rules(
        [
            Atom.first(Var.X)[10,] <= (Atom.my_custom_edge(Var.Y, Var.X), Atom.my_custom_features(Var.Y)),
        ]
    )


In [4]:
print(template)

{10} first(X) :- my_custom_edge(Y, X), my_custom_features(Y).


In [5]:
module_list = translate_rules(template.template)
print(module_list.modules)

In [6]:
temp_template = Template(module_list=module_list)
print(temp_template)

{10, 1} first(X) :- my_custom_features(Y), my_custom_edge(X, Y). [activation=identity, aggregation=sum]
first/1 [activation=identity]


In [7]:
temp_template.build(Backend.PYG, native_backend_models=True)

NeuraLogic(
  (module_list): ModuleList(
    (0): GCNConv(1, 10)
  )
)

# Simple GCN (two layers + embedding)

In [8]:
with Template().context() as template:
    template.add_rules(
        [
            Atom.my_custom_features(Var.A) <= Atom.f(Var.A),
            Atom.second(Var.X)[10,] <= (Atom.my_custom_edge(Var.Y, Var.X), Atom.first(Var.Y)),
            Atom.first(Var.X)[10,] <= (Atom.my_custom_edge(Var.Y, Var.X), Atom.my_custom_features(Var.Y)),
        ]
    )


In [9]:
print(template)

my_custom_features(A) :- f(A).
{10} second(X) :- my_custom_edge(Y, X), first(Y).
{10} first(X) :- my_custom_edge(Y, X), my_custom_features(Y).


In [10]:
module_list = translate_rules(template.template)
temp_template = Template(module_list=module_list)
print(temp_template)

{1, 1} my_custom_features(X) :- f(X). [activation=identity]
my_custom_features/1 [activation=identity]
{10, 1} first(X) :- my_custom_features(Y), my_custom_edge(X, Y). [activation=identity, aggregation=sum]
first/1 [activation=identity]
{10, 1} second(X) :- first(Y), my_custom_edge(X, Y). [activation=identity, aggregation=sum]
second/1 [activation=identity]


In [11]:
temp_template.build(Backend.PYG, native_backend_models=True)

NeuraLogic(
  (module_list): ModuleList(
    (0): Embedding(1, 1)
    (1): GCNConv(1, 10)
    (2): GCNConv(1, 10)
  )
)

# Simple SAGE

In [12]:
with Template().context() as template:
    template.add_rules(
        [
            Atom.first(Var.X)[10,] <= (Atom.my_custom_edge(Var.Y, Var.X), Atom.my_custom_features(Var.Y)),
            Atom.first(Var.X)[10,] <= (Atom.my_custom_features(Var.X)),
        ]
    )


In [13]:
print(template)

{10} first(X) :- my_custom_edge(Y, X), my_custom_features(Y).
{10} first(X) :- my_custom_features(X).


In [14]:
module_list = translate_rules(template.template)
temp_template = Template(module_list=module_list)
print(temp_template)

{10, 1} first(X) :- my_custom_features(Y), my_custom_edge(X, Y). [activation=identity, aggregation=avg]
{10, 1} first(X) :- my_custom_features(X). [activation=identity]
first/1 [activation=identity]


In [15]:
temp_template.build(Backend.PYG, native_backend_models=True)

NeuraLogic(
  (module_list): ModuleList(
    (0): SAGEConv(1, 10)
  )
)

# Mess test (embedding, sage, gcn all mixed)

In [20]:
with Template().context() as template:
    template.add_rules(
        [
            Atom.first(Var.X)[10,] <= (Atom.my_custom_edge(Var.Y, Var.X), Atom.my_custom_features(Var.Y)),
            Atom.my_custom_features(Var.A) <= Atom.f(Var.A),  # Embedding between sage
            Atom.second(Var.Y)[10,] <= (Atom.my_custom_edge(Var.Y, Var.X), Atom.first(Var.Y)),  # GCN between sage
            Atom.first(Var.X)[10,] <= (Atom.my_custom_features(Var.X)),
        ]
    )


In [21]:
print(template)

{10} first(X) :- my_custom_edge(Y, X), my_custom_features(Y).
my_custom_features(A) :- f(A).
{10} second(Y) :- my_custom_edge(Y, X), first(Y).
{10} first(X) :- my_custom_features(X).


In [22]:
module_list = translate_rules(template.template)
temp_template = Template(module_list=module_list)
print(temp_template)

{1, 1} my_custom_features(X) :- f(X). [activation=identity]
my_custom_features/1 [activation=identity]
{10, 1} first(X) :- my_custom_features(Y), my_custom_edge(X, Y). [activation=identity, aggregation=avg]
{10, 1} first(X) :- my_custom_features(X). [activation=identity]
first/1 [activation=identity]
{10, 1} second(X) :- first(Y), my_custom_edge(X, Y). [activation=identity, aggregation=sum]
second/1 [activation=identity]


In [23]:
temp_template.build(Backend.PYG, native_backend_models=True)

NeuraLogic(
  (module_list): ModuleList(
    (0): Embedding(1, 1)
    (1): SAGEConv(1, 10)
    (2): GCNConv(1, 10)
  )
)